# Практикум 11

1. Написать функцию validateCustomers(customer), которая принимает на входе список кортежей:
* Имя
* Фамилия
* Дата рождения
* Номер банковского счета (iban)

Функция возвращает мапу, где ключом является строка имя+фамилия, а значением - список сообщений об ошибках, возникших при валидации пользователя по следующим правилам:
Имя и фамилия не должны быть пустыми.
Возраст валиден, если он больше 18 лет. 
Iban должен соответствовать стандарту длины (начинаться с кода страны и содержать правильное количество символов)

Решение должно использовать исключения и итераторы. 


2. В исходных данных есть дата рождения, но правило проверки возраста требует проверку возраста. Нужно реализовать разницу между текущей датой и датой рождения клиента с целью проверки его возраста. 


3. Нужно создать свои исключения на каждый тип ошибки. При обнаружении ошибки бросается исключение с причиной ошибки, например, “возраст меньше 18 лет (дата рождения такая-то)” или “номер банковского счета неверен: когда страны “SR” не существует”. 
Ознакомьтесь с тем, что такое IBAN и реализуйте проверку нескольких правил формата (страна, длина).

4. Полученную из функции мапу нужно распечатать в формате:
Имя фамилия 1:

сообщение об ошибке 1

сообщение об ошибке 2

Имя фамилия 2:

	сообщение об ошибке 3
 
	сообщение об ошибке 4

И так далее. 


5. Реализовать чтение информации о клиентах из json файла и превращение его в список кортежей, который уже будет отдаваться на вход функции validateCustomers. Учесть, что json файл может не существовать на диске и в этом случае программа должна завершится с соответствующим правильным сообщением об ошибке (нужно перехватить соответствующее исключение). 

Формат json файла с информацией о пользователях составить самостоятельно, на свое усмотрение. Например:
`
[ 
{“firstName”: “Ivan”,
“lastName”: “Petrov”,
“dateOfBirth”: “15-03-2017”,
“iban”:”DE1234 0000 0000 01011 333”
 },
{и так далее}
]
`

6. Ответьте на вопрос: достаточно ли вам одной функции (validateCustomers) или есть смысл сделать несколько на проверку каждого правила?

In [7]:
# Пример использования
customers = [
    ("John", "Doe", "2000-01-01", "DE89370400440532013000"),
    ("Jane", "Smith", "1995-05-15", "FR7630006000011234567890189"),
    ("", "Brown", "1990-08-20", "DE89370400440532013000"),
    ("Alice", "Johnson", "2010-12-30", "INVALID_IBAN"),
]

validation_results = validateCustomers(customers)
for key, errors in validation_results.items():
    print(f"{key}: {errors}")

John Doe: []
Jane Smith: []
 Brown: ['Имя и фамилия не должны быть пустыми.']
Alice Johnson: ['Возраст должен быть больше 18 лет.']


In [36]:
import datetime

def validateCustomers(customers):
    validation_results = {}

    for customer in customers:
        name, surname, birthdate_str, iban = customer
        key = name +' '+ surname
        errors = []
        
        # Проверка имени и фамилии
        name_error = validate_name(name, surname)
        if name_error:
            errors.append(name_error)

        # Проверка даты рождения
        age_error = validate_age(birthdate_str)
        if age_error:
            errors.append(age_error)


        # Проверка IBAN
        iban_error = validate_iban(iban)
        if iban_error:
            errors.append(iban_error)

        validation_results[key] = errors

    return validation_results


# Функция для проверки имени и фамилии
def validate_name(name, surname):
    if not name: #name==''
        return 'Имя не должно быть пустым'
    if not surname:
        return 'Фамилия не должна быть пустой'
    return None

# Функция для расчета возраста
def calculate_age(birthdate_str): #"1995-05-15"
    year, month, day = list(map(int, birthdate_str.split('-')))

    # Узнаем текущую 
    current_day = datetime.datetime.now().day
    current_month = datetime.datetime.now().month
    current_year = datetime.datetime.now().year

    age = current_year - year

    if month > current_month and day > current_day: #(month, day) > (current_month, current_day):
        age -= 1

    return age

#Функция для проверки возраста
def validate_age(birthdate_str):
    age = calculate_age(birthdate_str)
    if age < 18:
        return f'Возраст меньше 18 (дата рождения {birthdate_str})'
    return None

# Проверка банковского номера
def validate_iban(iban):
    if not (15 <= len(iban) <= 34):
        return f'Некорректный формат IBAN {iban}'
    return None


        

In [38]:
# Пример использования
customers = [
    ("John", "Doe", "2000-01-01", "DE89370400440532013000"),
    ("Jane", "Smith", "1995-05-15", "FR7630006000011234567890189"),
    ("", "Brown", "1990-08-20", "DE89370400440532013000"),
    ("Alice", "Johnson", "2010-12-30", "INVALID_IBAN"),
]

validation_results = validateCustomers(customers)
print(validation_results)
#for key, errors in validation_results.items():
#    print(f"{key}: {errors}")

{'John Doe': [], 'Jane Smith': [], ' Brown': ['Имя не должно быть пустым'], 'Alice Johnson': ['Возраст меньше 18 (дата рождения 2010-12-30)', 'Некорректный формат IBAN INVALID_IBAN']}


In [41]:
import json

def read_customers_from_json(file_path):
    try:
        with open(file_path, 'r', encoding = 'utf-8') as file:
            customers_data = json.load(file)
    except FileNotFoundError:
        print(f'Файл не найден {file_path}')

    customers = []
    customer_iterator = iter(customers_data)

    while True:
        try: 
            customer = next(customer_iterator)
            name = customer.get('firstName','')
            surname = customer.get('lastName','')
            birthdate_str = customer.get('dateOfBirth','')
            iban = customer.get('iban','')
            customers.append((name, surname, birthdate_str, iban))
        except StopIteration:
            break
    return customers




In [44]:
def main():
    file_path = 'C:\\Users\\dplog\\Python\\data\\customers.json'
    try:
        customers = read_customers_from_json(file_path)
        validation_results = validateCustomers(customers)
        for key, errors in validation_results.items():
            print(f'{key}:')
            for error in errors:
                print(f'{error}')
    except FileNotFoundError as e:
        print(e)


if __name__=='__main__':
    main()

Ivan Petrov:
Anna Smith:
John Doe:
Alice Johnson:
Некорректный формат IBAN INVALID_IBAN
